In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
import pandas as pd
import string

In [39]:
data = pd.read_csv('/content/drive/MyDrive/Sheng Ye/fb-wx-data.csv')

In [40]:
data = data[['content', 'split', 'baidu_seg', 'baidu_prob', 'label']]

# Process 'split'

In [41]:
def split_text(text):
    length = len(text)
    text_block = []
    i, j = 0, 500

    while j < length:
        while text[j-1] != '。' and text[j-1] != '！':
            j -= 1
            if j == i:
                j = i + 500
                while text[j-1] != '，':
                    j -= 1
                    if j == i:
                        j = i + 500
                        break
                break
        text_block.append(text[i:j])
        i = j
        j = i + 500
    text_block.append(text[i:length])

    return text_block

In [42]:
for i in range(len(data.index)):
    data.at[i, 'split'] = split_text(data.at[i, 'content'])

# Process 'baidu_seg'

In [43]:
def seg_to_list(text):
    length = len(text)
    list_all = []
    temp_list = []

    i, j = 1, 1
    while j < length - 1:
        if text[i] == "'":
            j += 2
            while j < length - 1:
                if text[j] == "'" and (text[j+1] == ',' or text[j+1] == ']'):
                    temp_list.append(text[i+1:j])
                    i = j
                    break
                j += 1

        if text[j] == ']' and (text[j+1] == ',' or text[j+1] == ']'):
            list_all.append(temp_list)
            temp_list = []
            if text[j+1] == ']':
                break
            else: 
                i = j + 3
                j = i

        i += 1
        j += 1
    
    return list_all

In [44]:
for i in range(len(data.index)):
    data.at[i, 'baidu_seg'] = seg_to_list(data.at[i, 'baidu_seg'])

In [45]:
# Remove stopwords
stopwords = [line.strip() for line in open('/content/drive/MyDrive/Sheng Ye/stopwords.txt', encoding = 'UTF-8').readlines()]

In [46]:
def remove_stopwords(wordlist):
    output = [word for word in wordlist if word not in stopwords]
    return output

In [47]:
for i in range(len(data.index)):
    data.at[i, 'baidu_seg'] = [remove_stopwords(text) for text in data.at[i, 'baidu_seg']]

# Process 'baidu_prob'

In [48]:
def prob_to_list(text):
    length = len(text)
    list_all = []

    i, j = 0, 0
    while j < length - 1:
        if text[i].isdigit():
            while j < length:
                if text[j] == ',' or text[j] == ']':
                    list_all.append(float(text[i:j]))
                    i = j
                    break
                j += 1
        
        i += 1
        j += 1
    
    return list_all

In [49]:
for i in range(len(data.index)):
    data.at[i, 'baidu_prob'] = prob_to_list(data.at[i, 'baidu_prob'])

# View Data

In [50]:
data.head()

,content,split,baidu_seg,baidu_prob,label
0,本网讯（通讯员孙静娴）12月13日上午，中建一局房地产公司组织召开第七次中心组学习（扩大）会...,[本网讯（通讯员孙静娴）12月13日上午，中建一局房地产公司组织召开第七次中心组学习（扩大）...,"[[网讯, 通讯员, 孙静娴, 12月13日上午, 中建一局房地产公司, 组织, 召开, 第...","[0.00307655, 0.00122125, 0.00661374]",0.0
1,原标题：三项制度改革让“能者上、平者让、庸者下”11月11日，中建一局二公司生产组织中心主任...,[原标题：三项制度改革让“能者上、平者让、庸者下”11月11日，中建一局二公司生产组织中心主...,"[[原, 标题, 三项, 制度, 改革, 能者, 平者, 庸者, 11月11日, 中建一局二...","[0.240754, 0.918229, 0.0709911]",0.0
2,原标题：六家工地劳动用工标准受到通报表彰记者29日从郑州城乡建设局了解到，今年第四季度，有6...,[原标题：六家工地劳动用工标准受到通报表彰记者29日从郑州城乡建设局了解到，今年第四季度，有...,"[[原, 标题, 六家, 工地, 劳动, 用工, 标准, 通报, 表彰, 记者, 29日, ...",[0.0979948],0.0
3,合肥八院始建于1952年，俗称三康医院，已走过67年的辉煌历史，是一所集医疗、护理、保健、教...,[合肥八院始建于1952年，俗称三康医院，已走过67年的辉煌历史，是一所集医疗、护理、保健、...,"[[合肥八院, 始建于, 1952年, 俗称, 三康医院, 走过, 67年, 辉煌, 历史,...","[0.100506, 0.1208]",0.0
4,打造创新力，加强竞争力，提升发展力——近年来，中建一局二公司在全面加强党的领导的同时，立足企...,[打造创新力，加强竞争力，提升发展力——近年来，中建一局二公司在全面加强党的领导的同时，立足...,"[[打造, 创新, 力, 加强, 竞争力, 提升, 发展, 力, 近年, 中建一局二公司, ...","[0.0123384, 0.514408, 0.28322]",0.0


In [52]:
data.loc[15982]

content       各地也纷纷出台配套意见。过程结算要求：过程结算文件直接作为竣工结算依据；先结算，后竣工；不结...
split         [各地也纷纷出台配套意见。过程结算要求：过程结算文件直接作为竣工结算依据；先结算，后竣工；不...
baidu_seg     [[各地, 纷纷, 出台, 配套, 意见, 过程, 结算, 要求, 过程, 结算, 文件, ...
baidu_prob    [0.991483, 0.00862393, 0.047756, 0.00427403, 0...
label                                                       NaN
Name: 15982, dtype: object

# Simplify

In [54]:
seg_list = []
for i in range(len(data.index)):
    for text in data.at[i, 'baidu_seg']:
        seg_list.append(text)

In [55]:
prob_list = []
for i in range(len(data.index)):
    for prob in data.at[i, 'baidu_prob']:
        prob_list.append(prob)

In [56]:
data_sim = pd.DataFrame({'baidu_seg': seg_list, 'baidu_prob': prob_list})

In [57]:
len(data_sim.index)

66760

In [58]:
data_sim.tail(10)

,baidu_seg,baidu_prob
66750,"[7, 投资, 控制, 未见, 合同, 情况, 不明, 一点, 需要, 声明, 工程, 变更...",0.545663
66751,"[现场, 电, 应该, 采取, 三项, 五线制, 采取, 三级, 配电, 两级, 保护, 做...",0.542713
66752,"[施工, 过程, 特别, 强调, 混凝土, 浇筑, 过程, 必须, 两级, 漏电, 保护, ...",0.036550
66753,"[各地, 纷纷, 出台, 配套, 意见, 过程, 结算, 要求, 过程, 结算, 文件, 直...",0.991483
66754,"[本次, 培训, 共有, 十, 部分, 内容, 一, 过程, 结算, 政策, 概要, 二, ...",0.008624
66755,"[七, 工程, 签证, 实施, 要点, 1, 签证, 类型, 2, 签证, 时间, 3., ...",0.047756
66756,"[汪老师, 自办, 案例, 独创, 体系, 精细, 研究, 切实, 有用, 生动, 有趣, ...",0.004274
66757,"[5人次, 套餐, 8折, 优惠, 10人次, 套餐, 7折, 优惠, 套餐, 期限, 不,...",0.049731
66758,"[二, 交通, 路线, 1, 咸阳国际机场, 西安名都国际酒店, 距离, 35公里, 公共交...",0.133649
66759,"[度川, 教育, 权威, 政策, 解读, 紧贴, 企业, 需求, 资深, 授课, 导师, 专...",0.000632
